In [ ]:
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
import gensim
from gensim import corpora
from spacy import displacy
import pyLDAvis.gensim_models
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

#warning
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("Resume.csv")
df = df.reindex(np.random.permutation(df.index))
data = df.copy().iloc[
    0:200,
]
data.head()

In [ ]:
nlp = spacy.load("en_core_web_lg")
skill_pattern_path = "jz_skill_patterns.jsonl"

In [ ]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

In [ ]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [ ]:
clean = []
for i in range(data.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        data["Resume_str"].iloc[i],
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean.append(review)

In [ ]:
data["Clean_Resume"] = clean
data["skills"] = data["Clean_Resume"].str.lower().apply(get_skills)
data["skills"] = data["skills"].apply(unique_skills)
data.head()

In [ ]:
input_resume="""DEEPAK S deepaksridar2@gmail.com ⋄ www.linkedin.com/in/deepak035 ⋄ https://github.com/Its-Deepak-S
 OBJECTIVE
 I am a budding ML Engineer who’s passionate about Deep Learning and Data Science. I am always looking for
 opportunities to utilize my skills and knowledge on real- world tasks and upskilling myself in the process.
 EDUCATION
 B.Tech CSE (Specialization in AIML), Jain(Deemed-to-be University)
 Class 12, Chavara Public School
 SKILLS
 Expected 2025
 2020-2021
 Technical Skills
 Soft Skills
 Languages
 EXPERIENCE
 Deep Learning, Python, Computer Vision, MySQL, Java, JavaScript
 Communication, Problem Solving, Teamwork, Interpersonal Skills
 English, Malayalam, Hindi
 Intern in Artificial Intelligence
 Corizo
 Intern in Full-stack Development
 Capabl
 PROJECTS
 05/23- 06/23
 06/23- 07/23
 Image Generation GAN: Image Generation GAN leverages Generative Adversarial Networks (GANs) on Fashion
 MNIST data to produce realistic fashion images, showcasing the power of deep learning in generating synthetic data
 with high fidelity and diversity.
 Resume Analyzer: A project that utilizes Natural Language Processing techniques to parse and analyze resumes,
 extracting key information such as skills, experience, and qualifications, streamlining the recruitment process with
 efficiency and accuracy
 Real-time Face and Eye Detection: A powerful computer vision project that utilizes advanced algorithms to
 detect and track faces and eyes in real-time video streams"""

In [ ]:
colors = {
    "Job-Category": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
    "SKILL": "linear-gradient(90deg, #9BE15D, #00E3AE)",
    "ORG": "#ffd966",
    "PERSON": "#e06666",
    "GPE": "#9fc5e8",
    "DATE": "#c27ba0",
    "ORDINAL": "#674ea7",
    "PRODUCT": "#f9cb9c",
}
options = {
    "ents": [
        "Job-Category",
        "SKILL",
        "ORG",
        "PERSON",
        "GPE",
        "DATE",
        "ORDINAL",
        "PRODUCT",
    ],
    "colors": colors,
}
sent = nlp(data["Resume_str"].iloc[5])
displacy.render(sent, style="ent", jupyter=True, options=options)

In [ ]:
sent2 = nlp(input_resume)
displacy.render(sent2, style="ent", jupyter=True, options=options)

In [ ]:
inp_skills="""Deep Learning, Data Science, AI, Computer Vision"""

In [ ]:
req_skills = inp_skills.lower().split(",")
resume_skills = unique_skills(get_skills(input_resume.lower()))
score = 0
for x in req_skills:
    if x in resume_skills:
        score += 1
req_skills_len = len(req_skills)
match = round(score / req_skills_len * 100, 1)

print(f"The current Resume is {match}% matched to your requirements")